In [ ]:
import sire as sr

In [ ]:
mols = sr.stream.load("bound_31_42.bss")
mols.add_shared_property("space", mols.property("space"))
timestep = "1fs"

In [ ]:
# link reference properties to main properties
for mol in mols.molecules("molecule property is_perturbable"):
    mols.update(mol.perturbation().link_to_reference().commit())

In [ ]:
ligand = mols["molecule with property is_perturbable"]
ligand

In [ ]:
ligand_center = ligand.evaluate().center()
radius = "15 A"

In [ ]:
restraints = sr.restraints.positional(
    mols,
    f"residues within {radius} of {ligand_center}",
    position=ligand_center,
    r0=radius,
    k="10 kcal mol-1 A-2",
)

In [ ]:
restraints

In [ ]:
mols = (
    mols.minimisation(
        fixed=f"not (residues within {radius} of {ligand_center})",
        restraints=restraints,
        map={"ignore_perturbations": True},
    )
    .run()
    .commit()
)

In [ ]:
d = mols.dynamics(
    timestep=timestep,
    temperature="25oC",
    restraints=restraints,
    fixed=f"not (residues within {radius} of {ligand_center})",
)

In [ ]:
d.run("200ps", save_frequency="1ps")

In [ ]:
mols = d.commit()

In [ ]:
sr.save(mols.trajectory()[0], "tyk2_bubble", format=["pdb"])

In [ ]:
sr.save(mols.trajectory(), "tyk2_bubble", format=["DCD"])